In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pickle
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import ImageNet, CIFAR10

from PyTorch_CIFAR10.cifar10_models.mobilenetv2 import mobilenet_v2
from PyTorch_CIFAR10.UTILS_TORCH import *

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Currently using: '{device}'")

Currently using: 'cuda'


In [3]:
# Hyperparameters
NUM_EPOCHS = 100
BATCH_SIZE = 64
LEARN_RATE = 0.001
ALPHA = 0
TEMP = 1
NUM_WORKERS = 16
OVERLAY_PROB = 0.25

In [4]:
IGS = "PyTorch_CIFAR10/Captum_IGs.npy"
print(f"Using IGs: {IGS}")

# Load the precomputed IGs
igs = np.load(IGS)
print(f"IGs shape: {igs.shape}")

Using IGs: PyTorch_CIFAR10/Captum_IGs.npy
IGs shape: (50000, 32, 32)


In [5]:
precomputed_logits = np.load("PyTorch_CIFAR10/data/cifar10_logits.npy")
print("Shape of teacher_logits:", precomputed_logits.shape)

Shape of teacher_logits: (50000, 10)


In [6]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),
    ]
)

student_aug = transforms.Compose(
    [
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]
)

In [7]:
# Define the full training dataset without augmentation for splitting
train_dataset = CIFAR10_KD(
    root="PyTorch_CIFAR10/data",
    train=True,
    transform=student_aug,
    precomputed_logits=precomputed_logits,
)

# Load the data into batches
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=False,
    persistent_workers=True,
)

# load student data
test_data = CIFAR10(
    root="PyTorch_CIFAR10/data",
    train=False,
    transform=transform,
)

test_loader = DataLoader(
    test_data,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=False,
    persistent_workers=True,
)

Files already downloaded and verified


In [8]:
Teacher = mobilenet_v2(pretrained=True)
Teacher.to(device);

In [9]:
teacher_params = count_parameters(Teacher)

In [10]:
Student = SmallerMobileNet(mobilenet_v2(pretrained=False))
Student.to(device);

In [11]:
student_params = count_parameters(Student)

In [12]:
teacher_params / student_params

4.115345410654685

# First Student

In [13]:
class Smaller(nn.Module):
    def __init__(self, original_model):
        super(Smaller, self).__init__()
        self.features = nn.Sequential(*list(original_model.features.children())[:-3])

        # Dynamically find the output channels of the last convolutional block used
        # Iterate backwards over the blocks to find the last convolutional layer
        for block in reversed(self.features):
            if hasattr(block, "conv"):
                # If the block contains a 'conv' attribute, likely to be a sequential module
                if hasattr(block.conv, "__iter__"):
                    # Find the last Conv2d module in the block
                    for layer in reversed(block.conv):
                        if isinstance(layer, nn.Conv2d):
                            num_output_channels = layer.out_channels
                            break
                    break
            elif isinstance(block, nn.Conv2d):
                num_output_channels = block.out_channels
                break

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_output_channels, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

In [14]:
Student_1 = Smaller(mobilenet_v2(pretrained=False))
Student_1.to(device);

In [15]:
S1 = count_parameters(Student_1)
teacher_params / S1

2.194111842040726

In [16]:
Student_1, acc_1 = train_eval_kd(
    student=Student_1,
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=NUM_EPOCHS,
    lr=LEARN_RATE,
    TEMP=TEMP,
    ALPHA=ALPHA,
    device=device,
    csv_path="PyTorch_CIFAR10/Histories/Compression/Compression_of_2.19_student.csv",
)

Epoch 1/100: Loss: 1.4995,            Accuracy: 45.30%


Epoch 2/100: Loss: 1.0723,            Accuracy: 62.07%


Epoch 3/100: Loss: 0.8560,            Accuracy: 69.80%


Epoch 4/100: Loss: 0.7239,            Accuracy: 74.92%


Epoch 5/100: Loss: 0.6338,            Accuracy: 78.10%


...

Epoch 96/100: Loss: 0.0538,            Accuracy: 98.12%


Epoch 97/100: Loss: 0.0472,            Accuracy: 98.33%


Epoch 98/100: Loss: 0.0489,            Accuracy: 98.26%


Epoch 99/100: Loss: 0.0509,            Accuracy: 98.25%


Epoch 100/100: Loss: 0.0488,            Accuracy: 98.31%


In [17]:
print(f"Test Acc = {acc_1:.2f}%")

Test Acc = 91.61%


# Second Student

In [18]:
class Smaller(nn.Module):
    def __init__(self, original_model):
        super(Smaller, self).__init__()
        self.features = nn.Sequential(*list(original_model.features.children())[:-7])

        # Dynamically find the output channels of the last convolutional block used
        # Iterate backwards over the blocks to find the last convolutional layer
        for block in reversed(self.features):
            if hasattr(block, "conv"):
                # If the block contains a 'conv' attribute, likely to be a sequential module
                if hasattr(block.conv, "__iter__"):
                    # Find the last Conv2d module in the block
                    for layer in reversed(block.conv):
                        if isinstance(layer, nn.Conv2d):
                            num_output_channels = layer.out_channels
                            break
                    break
            elif isinstance(block, nn.Conv2d):
                num_output_channels = block.out_channels
                break

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_output_channels, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

In [19]:
Student_2 = Smaller(mobilenet_v2(pretrained=False))
Student_2.to(device);

In [20]:
S2 = count_parameters(Student_2)
teacher_params / S2

7.286700938902897

In [21]:
Student_2, acc_2 = train_eval_kd(
    student=Student_2,
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=NUM_EPOCHS,
    lr=LEARN_RATE,
    TEMP=TEMP,
    ALPHA=ALPHA,
    device=device,
    csv_path="PyTorch_CIFAR10/Histories/Compression/Compression_of_7.29_student.csv",
)

Epoch 1/100: Loss: 1.4100,            Accuracy: 48.36%


Epoch 2/100: Loss: 0.9820,            Accuracy: 65.14%


Epoch 3/100: Loss: 0.7856,            Accuracy: 72.50%


Epoch 4/100: Loss: 0.6625,            Accuracy: 76.83%


Epoch 5/100: Loss: 0.5873,            Accuracy: 79.70%


...

Epoch 96/100: Loss: 0.0529,            Accuracy: 98.09%


Epoch 97/100: Loss: 0.0521,            Accuracy: 98.12%


Epoch 98/100: Loss: 0.0468,            Accuracy: 98.33%


Epoch 99/100: Loss: 0.0526,            Accuracy: 98.20%


Epoch 100/100: Loss: 0.0523,            Accuracy: 98.20%


In [22]:
print(f"Test Acc = {acc_2:.2f}%")

Test Acc = 90.94%


# Third Student

In [23]:
class Smaller(nn.Module):
    def __init__(self, original_model):
        super(Smaller, self).__init__()
        self.features = nn.Sequential(*list(original_model.features.children())[:-9])

        # Dynamically find the output channels of the last convolutional block used
        # Iterate backwards over the blocks to find the last convolutional layer
        for block in reversed(self.features):
            if hasattr(block, "conv"):
                # If the block contains a 'conv' attribute, likely to be a sequential module
                if hasattr(block.conv, "__iter__"):
                    # Find the last Conv2d module in the block
                    for layer in reversed(block.conv):
                        if isinstance(layer, nn.Conv2d):
                            num_output_channels = layer.out_channels
                            break
                    break
            elif isinstance(block, nn.Conv2d):
                num_output_channels = block.out_channels
                break

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_output_channels, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

In [24]:
Student_3 = Smaller(mobilenet_v2(pretrained=False))
Student_3.to(device);

In [25]:
S3 = count_parameters(Student_3)
teacher_params / S3

12.042134619733172

In [26]:
Student_3, acc_3 = train_eval_kd(
    student=Student_3,
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=NUM_EPOCHS,
    lr=LEARN_RATE,
    TEMP=TEMP,
    ALPHA=ALPHA,
    device=device,
    csv_path="PyTorch_CIFAR10/Histories/Compression/Compression_of_12.04_student.csv",
)

Epoch 1/100: Loss: 1.3818,            Accuracy: 49.81%


Epoch 2/100: Loss: 0.9780,            Accuracy: 65.35%


Epoch 3/100: Loss: 0.8048,            Accuracy: 71.96%


Epoch 4/100: Loss: 0.6887,            Accuracy: 75.84%


Epoch 5/100: Loss: 0.6192,            Accuracy: 78.48%


...

Epoch 96/100: Loss: 0.0665,            Accuracy: 97.65%


Epoch 97/100: Loss: 0.0698,            Accuracy: 97.51%


Epoch 98/100: Loss: 0.0741,            Accuracy: 97.45%


Epoch 99/100: Loss: 0.0686,            Accuracy: 97.50%


Epoch 100/100: Loss: 0.0664,            Accuracy: 97.65%


In [27]:
print(f"Test Acc = {acc_3:.2f}%")

Test Acc = 90.36%


# Fourth Student

In [28]:
class Smaller(nn.Module):
    def __init__(self, original_model):
        super(Smaller, self).__init__()
        self.features = nn.Sequential(*list(original_model.features.children())[:-11])

        # Dynamically find the output channels of the last convolutional block used
        # Iterate backwards over the blocks to find the last convolutional layer
        for block in reversed(self.features):
            if hasattr(block, "conv"):
                # If the block contains a 'conv' attribute, likely to be a sequential module
                if hasattr(block.conv, "__iter__"):
                    # Find the last Conv2d module in the block
                    for layer in reversed(block.conv):
                        if isinstance(layer, nn.Conv2d):
                            num_output_channels = layer.out_channels
                            break
                    break
            elif isinstance(block, nn.Conv2d):
                num_output_channels = block.out_channels
                break

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_output_channels, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

In [29]:
Student_4 = Smaller(mobilenet_v2(pretrained=False))
Student_4.to(device);

In [30]:
S4 = count_parameters(Student_4)
teacher_params / S4

28.974816695598104

In [31]:
Student_4, acc_4 = train_eval_kd(
    student=Student_4,
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=NUM_EPOCHS,
    lr=LEARN_RATE,
    TEMP=TEMP,
    ALPHA=ALPHA,
    device=device,
    csv_path="PyTorch_CIFAR10/Histories/Compression/Compression_of_28.97_student.csv",
)

Epoch 1/100: Loss: 1.4417,            Accuracy: 46.77%


Epoch 2/100: Loss: 1.0425,            Accuracy: 62.73%


Epoch 3/100: Loss: 0.8886,            Accuracy: 68.40%


Epoch 4/100: Loss: 0.7895,            Accuracy: 71.95%


Epoch 5/100: Loss: 0.7187,            Accuracy: 74.88%


...

Epoch 96/100: Loss: 0.1622,            Accuracy: 94.20%


Epoch 97/100: Loss: 0.1680,            Accuracy: 93.94%


Epoch 98/100: Loss: 0.1627,            Accuracy: 94.25%


Epoch 99/100: Loss: 0.1598,            Accuracy: 94.23%


Epoch 100/100: Loss: 0.1570,            Accuracy: 94.25%


In [32]:
print(f"Test Acc = {acc_4:.2f}%")

Test Acc = 87.31%


# Fifth Student

In [33]:
class Smaller(nn.Module):
    def __init__(self, original_model):
        super(Smaller, self).__init__()
        self.features = nn.Sequential(*list(original_model.features.children())[:-13])

        # Dynamically find the output channels of the last convolutional block used
        # Iterate backwards over the blocks to find the last convolutional layer
        for block in reversed(self.features):
            if hasattr(block, "conv"):
                # If the block contains a 'conv' attribute, likely to be a sequential module
                if hasattr(block.conv, "__iter__"):
                    # Find the last Conv2d module in the block
                    for layer in reversed(block.conv):
                        if isinstance(layer, nn.Conv2d):
                            num_output_channels = layer.out_channels
                            break
                    break
            elif isinstance(block, nn.Conv2d):
                num_output_channels = block.out_channels
                break

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_output_channels, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

In [34]:
Student_5 = Smaller(mobilenet_v2(pretrained=False))
Student_5.to(device);

S5 = count_parameters(Student_5)
teacher_params / S5

54.5931657310227

In [35]:
Student_5, acc_5 = train_eval_kd(
    student=Student_5,
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=NUM_EPOCHS,
    lr=LEARN_RATE,
    TEMP=TEMP,
    ALPHA=ALPHA,
    device=device,
    csv_path="PyTorch_CIFAR10/Histories/Compression/Compression_of_54.59_student.csv",
)

Epoch 1/100: Loss: 1.4645,            Accuracy: 46.79%


Epoch 2/100: Loss: 1.1411,            Accuracy: 59.17%


Epoch 3/100: Loss: 1.0083,            Accuracy: 64.31%


Epoch 4/100: Loss: 0.9225,            Accuracy: 67.30%


Epoch 5/100: Loss: 0.8599,            Accuracy: 69.53%


...

Epoch 96/100: Loss: 0.2744,            Accuracy: 90.29%


Epoch 97/100: Loss: 0.2786,            Accuracy: 90.18%


Epoch 98/100: Loss: 0.2775,            Accuracy: 90.11%


Epoch 99/100: Loss: 0.2760,            Accuracy: 90.27%


Epoch 100/100: Loss: 0.2745,            Accuracy: 90.33%


In [36]:
print(f"Test Acc = {acc_5:.2f}%")

Test Acc = 84.97%


# Sixth Student

In [37]:
class Smaller(nn.Module):
    def __init__(self, original_model):
        super(Smaller, self).__init__()
        self.features = nn.Sequential(*list(original_model.features.children())[:-15])

        # Dynamically find the output channels of the last convolutional block used
        # Iterate backwards over the blocks to find the last convolutional layer
        for block in reversed(self.features):
            if hasattr(block, "conv"):
                # If the block contains a 'conv' attribute, likely to be a sequential module
                if hasattr(block.conv, "__iter__"):
                    # Find the last Conv2d module in the block
                    for layer in reversed(block.conv):
                        if isinstance(layer, nn.Conv2d):
                            num_output_channels = layer.out_channels
                            break
                    break
            elif isinstance(block, nn.Conv2d):
                num_output_channels = block.out_channels
                break

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_output_channels, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

In [38]:
Student_6 = Smaller(mobilenet_v2(pretrained=False))
Student_6.to(device);

S6 = count_parameters(Student_6)
teacher_params / S6

139.426630095998

In [39]:
Student_6, acc_6 = train_eval_kd(
    student=Student_6,
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=NUM_EPOCHS,
    lr=LEARN_RATE,
    TEMP=TEMP,
    ALPHA=ALPHA,
    device=device,
    csv_path="PyTorch_CIFAR10/Histories/Compression/Compression_of_139.43_student.csv",
)

Epoch 1/100: Loss: 1.6001,            Accuracy: 41.67%


Epoch 2/100: Loss: 1.2686,            Accuracy: 54.49%


Epoch 3/100: Loss: 1.1331,            Accuracy: 59.87%


Epoch 4/100: Loss: 1.0615,            Accuracy: 62.06%


Epoch 5/100: Loss: 1.0094,            Accuracy: 64.13%


...

Epoch 96/100: Loss: 0.5562,            Accuracy: 80.54%


Epoch 97/100: Loss: 0.5520,            Accuracy: 80.71%


Epoch 98/100: Loss: 0.5563,            Accuracy: 80.55%


Epoch 99/100: Loss: 0.5541,            Accuracy: 80.72%


Epoch 100/100: Loss: 0.5537,            Accuracy: 80.48%


In [40]:
print(f"Test Acc = {acc_6:.2f}%")

Test Acc = 76.46%


# Seventh Student

In [41]:
class Smaller(nn.Module):
    def __init__(self, original_model):
        super(Smaller, self).__init__()
        self.features = nn.Sequential(*list(original_model.features.children())[:-17])

        # Dynamically find the output channels of the last convolutional block used
        # Iterate backwards over the blocks to find the last convolutional layer
        for block in reversed(self.features):
            if hasattr(block, "conv"):
                # If the block contains a 'conv' attribute, likely to be a sequential module
                if hasattr(block.conv, "__iter__"):
                    # Find the last Conv2d module in the block
                    for layer in reversed(block.conv):
                        if isinstance(layer, nn.Conv2d):
                            num_output_channels = layer.out_channels
                            break
                    break
            elif isinstance(block, nn.Conv2d):
                num_output_channels = block.out_channels
                break

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_output_channels, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

In [42]:
Student_7 = Smaller(mobilenet_v2(pretrained=True))
Student_7.to(device);

S7 = count_parameters(Student_7)
teacher_params / S7

1121.7061183550652

In [43]:
Student_7, acc_7 = train_eval_kd(
    student=Student_7,
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=NUM_EPOCHS,
    lr=LEARN_RATE,
    TEMP=TEMP,
    ALPHA=ALPHA,
    device=device,
    csv_path="PyTorch_CIFAR10/Histories/Compression/Compression_of_1121.71_student.csv",
)

Epoch 1/100: Loss: 1.9538,            Accuracy: 27.04%


Epoch 2/100: Loss: 1.6797,            Accuracy: 36.25%


Epoch 3/100: Loss: 1.6098,            Accuracy: 40.00%


Epoch 4/100: Loss: 1.5624,            Accuracy: 42.72%


Epoch 5/100: Loss: 1.5329,            Accuracy: 44.03%


...

Epoch 96/100: Loss: 1.2623,            Accuracy: 55.06%


Epoch 97/100: Loss: 1.2586,            Accuracy: 55.12%


Epoch 98/100: Loss: 1.2620,            Accuracy: 55.12%


Epoch 99/100: Loss: 1.2589,            Accuracy: 55.36%


Epoch 100/100: Loss: 1.2632,            Accuracy: 54.83%


In [44]:
print(f"Test Acc = {acc_7:.2f}%")

Test Acc = 49.35%


# Normal Student

In [45]:
Student_8 = SmallerMobileNet(mobilenet_v2(pretrained=False))
Student_8.to(device)

S8 = count_parameters(Student_8)
teacher_params / S8

4.115345410654685

In [46]:
Student_8, acc_8 = train_eval_kd(
    student=Student_8,
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=NUM_EPOCHS,
    lr=LEARN_RATE,
    TEMP=TEMP,
    ALPHA=ALPHA,
    device=device,
    csv_path="PyTorch_CIFAR10/Histories/Compression/Compression_of_4.12_student.csv",
)

Epoch 1/100: Loss: 1.4501,            Accuracy: 46.81%


Epoch 2/100: Loss: 1.0068,            Accuracy: 64.61%


Epoch 3/100: Loss: 0.7807,            Accuracy: 72.86%


Epoch 4/100: Loss: 0.6597,            Accuracy: 77.22%


Epoch 5/100: Loss: 0.5855,            Accuracy: 79.67%


...

Epoch 96/100: Loss: 0.0482,            Accuracy: 98.35%


Epoch 97/100: Loss: 0.0460,            Accuracy: 98.33%


Epoch 98/100: Loss: 0.0536,            Accuracy: 98.14%


Epoch 99/100: Loss: 0.0456,            Accuracy: 98.35%


Epoch 100/100: Loss: 0.0470,            Accuracy: 98.33%


In [47]:
print(f"Test Acc = {acc_8:.2f}%")

Test Acc = 91.14%
